# tips dataset
1. Load the tips dataset.
    * Create a column named tip_percentage. This should be the tip amount divided by the total bill.

    * Create a column named price_per_person. This should be the total bill divided by the party size.

    * Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?

    * Use select k best and recursive feature elimination to select the top 2 features for predicting tip amount. What are they?

    * Use select k best and recursive feature elimination to select the top 2 features for predicting tip percentage. What are they?

    * Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

## #1

In [1]:
import pandas as pd

from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from pydataset import data

In [2]:
df = data('tips')
df.head(3), len(df)

(   total_bill   tip     sex smoker  day    time  size
 1       16.99  1.01  Female     No  Sun  Dinner     2
 2       10.34  1.66    Male     No  Sun  Dinner     3
 3       21.01  3.50    Male     No  Sun  Dinner     3,
 244)

Create a column named tip_percentage. This should be the tip amount divided by the total bill.

Create a column named price_per_person. This should be the total bill divided by the party size.

In [3]:
### New Columns ###
df['tip_percentage'] = df['tip'] / df['total_bill']
df['price_per_person'] = df['total_bill'] / df['size']
df.head(3)

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,3.446667
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,7.003333


Before using any of the methods discussed in the lesson, which features do you think would be most important for:
- predicting the tip amount?
    * total_bill, size, time
- predicting the tip percentage?
    * total_bill, tip, time

Use select k best and recursive feature elimination to select the top 2 features for predicting tip amount. What are they?

In [4]:
df['smoker'].unique()

array(['No', 'Yes'], dtype=object)

In [5]:
df['time'].unique()

array(['Dinner', 'Lunch'], dtype=object)

In [6]:
df['day'].unique()

array(['Sun', 'Sat', 'Thur', 'Fri'], dtype=object)

In [7]:
df['sex'] = df['sex'].map({'Male':0, 'Female':1})
df['smoker'] = df['smoker'].map({'No':0, 'Yes':1})
# df['day'] = df['day'].map({'Thur':0, 'Fri':1, 'Sat':2, 'Sun':3})
df['time'] = df['time'].map({'Lunch':0, 'Dinner':1})
df.head(3)

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,1,0,Sun,1,2,0.059447,8.495000
2,10.34,1.66,0,0,Sun,1,3,0.160542,3.446667
3,21.01,3.50,0,0,Sun,1,3,0.166587,7.003333


#### K-Best: target = 'tip'

In [8]:
train, test = train_test_split(df, test_size=0.2, random_state=123)
X_train, y_train = train.drop(columns=['tip', 'day']), train.tip
X_test, y_test = test.drop(columns=['tip', 'day']), test.tip

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled[:3]

array([[ 2.22751067, -0.74833148, -0.79915922,  0.59511904,  1.51285289,
         0.45004414,  0.67765502],
       [-0.4404687 , -0.74833148,  1.2513151 ,  0.59511904, -0.57939047,
        -1.03330878, -0.00598934],
       [-0.76989051,  1.33630621,  1.2513151 ,  0.59511904, -0.57939047,
         0.18120219, -0.50426744]])

In [10]:
kbest = SelectKBest(f_regression, k=2)
kbest.fit(X_train_scaled, y_train)

SelectKBest(k=2, score_func=<function f_regression at 0x7fd6f246f9d0>)

In [11]:
X_train.columns[kbest.get_support()]

Index(['total_bill', 'size'], dtype='object')